# Ejercicio Primer Parcial (Regresion logistica One vs All) 
 <h3>Primer Parcial Ejercicio D1 (Clasificacion one-vs-all) <h3>
 <HR>
 <h3>
  NOMBRE: Rojas Arroyo Luis Fernando <br>
  CARRERA: INGENIERIA DE SISTEMAS <BR>
  FECHA: 09/04/2024 <BR>

  <!-- * [Enlace de invitacion para ser colaborador](https://github.com/bspoloo/SIS420-012024/invitations)
  
  * [Enlace al git hub](https://github.com/bspoloo/SIS420-012024/tree/main/Laboratorios/Laboratorio%203)
  
  * [Enlace al Colab](https://colab.research.google.com/github/bspoloo/SIS420-012024/blob/main/Laboratorios/Laboratorio%203/Laboratorio%203.ipynb?hl=es) -->
  
 <h3>

 Se hizo uso del para entrenar el modelo de **Regresion Logistica one vs all**

 A lo largo de los años, se han recopilado datos de "INVESTIGACIÓN INCENDIOS FORESTALES EN TURQUÍA" Haciendo uso del siguiente dataset: [THE CONNECTION BETWEEN PLANTS AND HOTPOINTS/TURKEY](https://www.kaggle.com/datasets/brsdincer/the-connection-between-plants-and-hotpointsturkey).

El archivo `dataset_01.csv` CONTIENE LOS DATOS NECESARIOS PARA INVESTIGAR LOS ÚLTIMOS INCENDIOS FORESTALES EN TURQUÍA.

Datos sobre los recientes incendios forestales en Turquía, publicados con permiso del Portal de la NASA.
Los datos se crearon a partir de los puntos calientes y se obtuvieron del satélite.

## 1. Clasificacion multiclase

Importamos todas las librerias necesarias para el manejo del dataset.

In [7]:
# utilizando la libreria os para manejos de directorios
import os

# Computacion vectorial y cientifica para python
import numpy as np

#importamos pandas para el manejo del dataset, y separarlos dentro de una matriz
import pandas as pd

#esta tabulate nos sirve para hacer tablas
from tabulate import tabulate

# Librerias para graficación (trazado de gráficos)
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D  # -> Necesario para graficar superficies 3D

#Para separa el 20% y 80%
from sklearn.model_selection import train_test_split

# Modulo de optimizacion en scipy
from scipy import optimize

### 1.1 Cargamos los datos del dataset
cargamos los datos haciendo el uso de la libreria **Pandas** que  es una herramienta poderosa y versátil utilizada para manipulación y análisis de datos. Ofrece estructuras de datos flexibles y eficientes para trabajar con datos tabulares, como hojas de cálculo en Excel o tablas SQL.


In [9]:
#Haciendo uso de la libreria pandas para leer el dataset, delimitado por ","
dataset = pd.read_csv('dataset_01.csv', delimiter=',')

#eliminamos la primera columan que seria nuestro "id" ya que no nos sirve para el analisis
dataset = dataset.drop(dataset.columns[0], axis=1)

#ahora tenemos un dataframe
#Imprimimos en una tabla el dataset para hacer un analisis mas claro.
dataset

,LONGITUDE,BRIGHTNESS,SCAN,TRACK,ACQ_DATE,ACQ_TIME,SATELLITE,CONFIDENCE,BRIGHT_T31,FRP,DAYNIGHT,TYPE,Katman
0,35.35940,306.00,2.10,1.40,11/1/2000,815,Terra,60,294.40,13.20,D,0,Sürekli Sulanan Alanlar
1,35.17860,307.80,1.10,1.00,11/2/2000,857,Terra,66,296.00,5.80,D,0,Sulanmayan Tarım Alanları
2,35.04440,305.30,1.10,1.00,11/2/2000,857,Terra,46,293.50,4.30,D,0,Sulanmayan Tarım Alanları
3,35.19090,310.00,1.10,1.00,11/2/2000,857,Terra,69,296.10,7.30,D,0,Sulanmayan Tarım Alanları
4,31.41720,307.90,1.10,1.00,11/2/2000,857,Terra,0,290.40,8.00,D,2,Limanlar
...,...,...,...,...,...,...,...,...,...,...,...,...,...
211294,43.69891,323.01,3.75,1.81,8/9/2021,928,Aqua,35,295.72,82.09,D,0,Bitki Değişim Alanları
211295,43.68993,322.73,3.75,1.81,8/9/2021,928,Aqua,53,295.43,84.54,D,0,Geniş Yapraklı Ormanlar
211296,43.68782,324.40,4.50,1.95,8/9/2021,1106,Aqua,50,307.05,90.94,D,0,Bitki Değişim Alanları
211297,43.69128,324.88,4.50,1.95,8/9/2021,1106,Aqua,50,307.00,95.24,D,0,Bitki Değişim Alanları


mostrando la tabla nos damos cuenta que nuestra Y a predicir es el **Type**, para tener una mejor vision de la cantidad de clases que existe, se hizo el siguiente codigo:

donde separamos nuestra columna Y con `value_counts()` este método de pandas cuenta el número de veces que aparece cada valor único en la columna **Type** del DataFrame. Devuelve una Serie pandas donde los índices son los valores únicos de la columna **Type** y los valores son el recuento de ocurrencias de cada valor.

In [14]:
#para contar cuantas clases contiene nuestra columna y
class_counts = dataset["TYPE"].value_counts()

#para contar cuantas caracteristicas tiene nuestro dataset, obviamente con sin contar nuestra y, por eso lo dropeamos, tambien dropeamos la primera columna que no tiene nombre
feactures_counts =dataset.drop(['TYPE'], axis=1)
feactures_counts = feactures_counts.shape[1]

print(f"la cantidad de caracteristicas es: { feactures_counts}")
print(f'la cantidad de clases es: 4')

#mostramos la cantidad de clases tiene, y en que cantidad
class_counts

la cantidad de caracteristicas es: 12
la cantidad de clases es: 4


TYPE
0    197051
2     14163
3        85
Name: count, dtype: int64

#### 1.1.1 Analisis del dataset:

Hay 4 clases diferentes(tipo de punto caliente inferido): etiquetados como 0, 1, 2 y 3.

* La clase 0 parece ser la más común, seguida por la clase 2 y luego la clase 3.
* El recuento de cada clase es el siguiente:
    - Clase 0: 197051 puntos.
    - Clase 2: 14163 puntos.
    - Clase 3: 85 puntos.
* Este análisis de la distribución de los puntajes calientes inferidos en el conjunto de datos es útil para comprender la proporción de cada clase. Esto puede ser relevante al desarrollar un modelo de clasificación como la Regresión Logística one vs all.

In [15]:
#almacenamos la cantidad de caracteristicas en una variable
input_layer_size  = feactures_counts;

#almacenamos la cantidad de clases en una variable

#para este caso no es necesario cambiar los valores de la ultima clase, ya que este cuenta con 0,1,2,3,
# en caso de contener 1,2,3, 4  o simplememte suma 1 a la cantidad de clases.
num_labels = 4;

#### 1.1.2 Separacion de datos
Se debe tomar en cuenta que cada clase tiene su propia cantidad, por lo cual separarlos directamente en un 80% para entrenamiento y un 20% para test no seria tan efectivo, ya que puede que en el 80% hay mas datos de una clase que las otras, provocando que nuestro modelo no conozca mucho sobre esa clase, por lo cual debe separarse un 80% para entrenamiento y un 20% para pruebas de cada clase.

Para este hacemos uso de la libreria **train_test_split**

In [18]:
# hacemos uso del DataFrame llamado 'dataset' que contiene nuestros datos datos
# y es la columna que contiene las etiquetas de clase (en este caso, 'type')

#creamos una variable temporal que contentra toda la columna de 'TYPE'
y_temp = dataset['TYPE']

# Para la clase 0

#donde y_temp es igual a 0, separamos los datos en train_class_0, test_class_0
data_class_0 = dataset[y_temp == 0]
train_class_0, test_class_0 = train_test_split(data_class_0, test_size=0.2, random_state=42)

# Para la clase 1

# #donde y_temp es igual a 1, separamos los datos en train_class_1, test_class_1
# data_class_1 = dataset[y_temp == 1]
# train_class_1, test_class_1 = train_test_split(data_class_1, test_size=0.2, random_state=42)

# Para la clase 2

#donde y_temp es igual a 2, separamos los datos en train_class_2, test_class_2
data_class_2 = dataset[y_temp == 2]
train_class_2, test_class_2 = train_test_split(data_class_2, test_size=0.2, random_state=42)


# Para la clase 3

#donde y_temp es igual a 3, separamos los datos en train_class_3, test_class_3
data_class_3 = dataset[y_temp == 3]
train_class_3, test_class_3 = train_test_split(data_class_3, test_size=0.2, random_state=42)

haciendo conteo de separacion de datos:


In [19]:
print(f"Para la clase 0 tenemos una cantidad de: { data_class_0.shape[0]} donde el 80% es: {train_class_0.shape[0]} y el 20% es: {test_class_0.shape[0]}")
print(f"Para la clase 1 tenemos una cantidad de: { data_class_2.shape[0]} donde el 80% es: {train_class_2.shape[0]} y el 20% es: {test_class_2.shape[0]}")
print(f"Para la clase 2 tenemos una cantidad de: { data_class_3.shape[0]} donde el 80% es: {train_class_3.shape[0]} y el 20% es: {test_class_3.shape[0]}")
print(f"La cantidad total de datos es: {dataset.shape[0]}")

Para la clase 0 tenemos una cantidad de: 197051 donde el 80% es: 157640 y el 20% es: 39411
Para la clase 1 tenemos una cantidad de: 14163 donde el 80% es: 11330 y el 20% es: 2833
Para la clase 2 tenemos una cantidad de: 85 donde el 80% es: 68 y el 20% es: 17
La cantidad total de datos es: 211299


Separamos cada uno en sus X_train, y_train, X_test y y_test respectivos:

In [21]:
#para la parte de entrenamiento, separamos las caracteristicas de la etiqueta

#para la clase 0
X_train_class_0 = train_class_0.drop(['TYPE'], axis=1)
y_train_class_0 = train_class_0['TYPE']

#para la clase 1
X_train_class_2 = train_class_2.drop(['TYPE'], axis=1)
y_train_class_2 = train_class_2['TYPE']

#para la clase 2
X_train_class_3 = train_class_3.drop(['TYPE'], axis=1)
y_train_class_3 = train_class_3['TYPE']

#ahora para la parte de pruebas, separamos las caracteristicas de la etiqueta
#para la clase 0
X_test_class_0 = test_class_0.drop(['TYPE'], axis=1)
y_test_class_0 = test_class_0['TYPE']

#para la clase 1
X_test_class_2 = test_class_2.drop(['TYPE'], axis=1)
y_test_class_2 = test_class_2['TYPE']

#para la clase 2
X_test_class_3 = test_class_3.drop(['TYPE'], axis=1)
y_test_class_3 = test_class_3['TYPE']

ahora unimos todos en una sola matriz para X y y:
pero luego debemos mezclar los datos, haciendo uso de ``np.random.permutation(len(X))`` genera un arreglo de índices permutados aleatoriamente.
Luego, estos índices se usan para reorganizar tanto las características como las etiquetas de entrenamiento y prueba.

In [38]:
#separando los datos de entrenamiento y pruebas

#para los datos de entrenamiento
X_train = pd.concat([X_train_class_0, X_train_class_2, X_train_class_3]).values
y_train = pd.concat([y_train_class_0, y_train_class_2, y_train_class_3]).values


indices_train = np.random.permutation(len(X_train))
X_train = X_train[indices_train]
y_train = y_train[indices_train]
m_train = len(y_train)

#para los datos de pruebas
X_test = pd.concat([X_test_class_0, X_test_class_2, X_test_class_3]).values
y_test = pd.concat([y_test_class_0, y_test_class_2, y_test_class_3]).values

indices_test = np.random.permutation(len(X_test))
X_test = X_test[indices_test]
y_test = y_test[indices_test]
X=X_test
y=y_test
m_test = len(y_test)
print(X_train)

[[32.3184 325.0 1.1 ... 13.2 'D' 'Sürekli Sulanan Alanlar']
 [33.1177 322.3 1.1 ... 21.3 'D' 'Sulanmayan Tarım Alanları']
 [27.7339 333.2 1.3 ... 17.9 'D' 'Sürekli Sulanan Alanlar']
 ...
 [31.9552 302.1 1.7 ... 7.0 'D' 'Sulanmayan Tarım Alanları']
 [40.875 319.3 1.1 ... 8.7 'D' 'Sürekli Sulanan Alanlar']
 [31.851 335.2 1.0 ... 17.8 'D' 'Sulanmayan Tarım Alanları']]


Visualizamos algunos datos:

In [76]:
print('{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}'.format(
    'X[:,0]', 'X[:, 1]', 'X[:, 2]', 'X[:, 3]', 'X[:, 4]', 'X[:, 5]', 'X[:, 6]', 'X[:, 7]', 'X[:, 8]', 'X[:, 9]', 'X[:, 10]', 'X[:, 11]','Y'
))
print('-' * 200)

for i in range(10):
    print('{:10.4f}{:10.3f}{:10.2f}{:10.1f}{:10.9s}{:10.1f}{:10.10s}{:10d}{:10.2f}{:10.2f}{:10.2s}{:10.20s}{:10.10s}'.format(
        X_train[i, 0],
        X_train[i, 1],
        X_train[i, 2],
        X_train[i, 3],
        X_train[i, 4],
        X_train[i, 5],
        X_train[i, 6],
        X_train[i, 7],
        X_train[i, 8],
        X_train[i, 9],
        X_train[i, 10],
        X_train[i, 11],
        str(y_train[i])
    ))


#mostramos la cantidad de ejemplos
print(" ")
print('El 80% de ejemplos para entrenamiento son la cantidad de: {:.0f} de ejemplos'.format( len(X_train)))
print('El 20% de ejemplos para pruebas son la cantidad de: {:.0f} de ejemplos'.format(  len(X_test)))

    X[:,0]   X[:, 1]   X[:, 2]   X[:, 3]   X[:, 4]   X[:, 5]   X[:, 6]   X[:, 7]   X[:, 8]   X[:, 9]  X[:, 10]  X[:, 11]         Y
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
   32.3184   325.000      1.10       1.010/11/200     1110.0Aqua              72    308.30     13.20D         Sürekli Sulanan Alan0         
   33.1177   322.300      1.10       1.02/12/2018      851.0Terra             82    289.10     21.30D         Sulanmayan Tarım Ala0         
   27.7339   333.200      1.30       1.18/17/2003     1057.0Aqua              56    321.20     17.90D         Sürekli Sulanan Alan0         
   28.2703   304.700      1.20       1.111/18/200     1058.0Aqua              60    294.00      6.20D         Sürekli Sulanan Alan0         
   27.8197   319.100      1.10       1.110/3/2006     1107.0Aqua              69    306.30     11.20D   

### 1.2 Normalización de caracteristicas

Al visualizar los datos se puede observar que las caracteristicas tienen diferentes magnitudes, por lo cual se debe transformar cada valor en una escala de valores similares, esto con el fin de que el descenso por el gradiente pueda converger mas rapidamente. Se aplica la normalizacion esto debido a que los datos de las X estan a diferentes escalas.

Hacemos el uso de la siguiente funcion para normalizar los datos de las columnas X:

In [77]:
def  featureNormalize(X):
    X_norm = X.copy()

    #creamos un array de ceros con una longitud igual al número de columnas en el array X. La variable mu y sigma se inicializa como este array de ceros.
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    #Creamos el promedio de cada columna de X
    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    sigma[sigma == 0] = 1
    #normalizamos los datos con la siguiente formula
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

Normalizando las caracteristicas:

In [87]:
X_norm, mu, sigma= featureNormalize(X)
print(X_norm)

# imprimir todos las X_norm de datos solo 10
print('{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}'.format(
    'X[:,0]', 'X[:, 1]', 'X[:, 2]', 'X[:, 3]', 'X[:, 4]', 'X[:, 5]', 'X[:, 6]', 'X[:, 7]', 'X[:, 8]', 'X[:, 9]', 'X[:, 10]', 'X[:, 11]','Y'
))
print('-' * 200)

for i in range(10):
    print('{:10.4f}{:10.3f}{:10.2f}{:10.1f}{:10.9s}{:10.1f}{:10.10s}{:10d}{:10.2f}{:10.2f}{:10.2s}{:10.20s}{:10.10s}'.format(
        X_norm[i, 0],
        X_norm[i, 1],
        X_norm[i, 2],
        X_norm[i, 3],
        X_norm[i, 4],
        X_norm[i, 5],
        X_norm[i, 6],
        X_norm[i, 7],
        X_norm[i, 8],
        X_norm[i, 9],
        X_norm[i, 10],
        X_norm[i, 11],
        y_train[i]
    ))

NameError: name 'X' is not defined

### 1.3 Creacion de la funcion Sigmoide
También conocida como la función logística, es una función matemática que toma cualquier número real como entrada y devuelve un valor en el rango de 0 a 1. Donde nuestra **Z** es nuestra hipotesis.

Creando la funcion:

In [41]:
def sigmoid(z):
    z = np.array(z)

    g = np.zeros(z.shape)

    g = 1 / (1 + np.exp(-z))

    return g

### 1.4 Funcion de costo
Aplicando la teoria donde nuestra funcion recibira parametros como Theta, x, y y lamda_, donde lamda_ es nuestro parametro de regularizacion.

Donde la funcion nos devuelve un costo y nuestro gradiente.

In [85]:
def lrCostFunction(theta, X, y, lambda_):
    #creamos una variable m que contiene la longitud de y
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    #inicializamos J y grad
    J = 0
    grad = np.zeros(theta.shape)

    #calculamos h haciendo uso de la funcion sigmoid, donde h es nuestra hipotesis
    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
    
    grad = grad + (lambda_ / m) * temp

    return J, grad

### 1.5 Clasificacion One-vs-All
Aqui estariamos usando un ciclo for para iterar sobre cada una de las clases, luego haciendo uso de `optimize.minimize` que es un método de la biblioteca *scipy* que encuentra el mínimo de una función. En este caso, se trata de minimizar la función de costos de regresión logística `(lrCostFunction)`.

Los parámetros iniciales ``(initial_theta)``.
Una tupla que contiene los datos de entrenamiento ``(X)``, las etiquetas ``(y == c)``, y el parámetro de regularización ``(lambda_)``.
``jac=True`` indica que la función de coste devuelve tanto el coste como el gradiente.
El método de optimización ``('CG' significa Gradiente Conjugado).``
El diccionario de opciones ``(options)``  que se establece en 1000 para limitar el número máximo de iteraciones del optimizador.

In [86]:
def oneVsAll(X, y, num_labels, lambda_):
    # algunas variables utiles
    #m es la longitud de y, n es la cantidad de columnas en X
    m, n = X.shape

    # inicializamos la matriz de thetas con ceros, y con n+1 columnas que seria nuestro sesgo
    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    # iteramos sobre cada etiqueta (clase) y entrenamos un clasificador
    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 1000}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

Inicializamos nuestra lambda con valor de *0.005*, usamos la funcion de `oneVsAll` donde pasamos los parametros de *X_norm*, *num_labels* que seria la cantidad de clases que tenemos, y nuestro *lanbda* para asi obtener nuestros Thetas para cada clase.

In [44]:

lambda_ = 0.005
all_theta = oneVsAll(X_norm, y_train, num_labels, lambda_)
print(all_theta.shape)

NameError: name 'X_norm' is not defined

In [16]:
for i in range(all_theta.shape[1]):
    print("Thetha",i," para la clase 0:", all_theta[0,i]," ; ", "Thetha",i," para la clase 1:", all_theta[1,i]," ; ", "Thetha",i," para la clase 2:", all_theta[2,i])

Thetha 0  para la clase 0: -1.081543148619021  ;  Thetha 0  para la clase 1: 0.14285348465754039  ;  Thetha 0  para la clase 2: -2.371132391239196
Thetha 1  para la clase 0: 0.0014111350123358936  ;  Thetha 1  para la clase 1: 0.016860051168269403  ;  Thetha 1  para la clase 2: -0.03736392896826251
Thetha 2  para la clase 0: -0.06071940308801994  ;  Thetha 2  para la clase 1: 0.007170383640216165  ;  Thetha 2  para la clase 2: 0.04980243075604156
Thetha 3  para la clase 0: 0.007170501633783687  ;  Thetha 3  para la clase 1: -0.0011672846659346171  ;  Thetha 3  para la clase 2: -0.014865592155955183
Thetha 4  para la clase 0: -0.02492779318261062  ;  Thetha 4  para la clase 1: 0.0211529044503502  ;  Thetha 4  para la clase 2: -0.001077602282414906
Thetha 5  para la clase 0: -0.09457652584528718  ;  Thetha 5  para la clase 1: -0.004293661515637002  ;  Thetha 5  para la clase 2: 0.1232058635687482
Thetha 6  para la clase 0: -0.020546861565206725  ;  Thetha 6  para la clase 1: 0.1597287781

#### 1.5.1 Prediccion One-vs-all
Aqui creamos solo la funcion, donde mandamos los parametros de `all_theta` y la `X` que en este caso puede ser las X de prueba, pero antes deben de estar normalizadas.

In [17]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # añadimos unos a la matriz de X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

### 1.6 Validaciones

Para este caso de validacion se hizo uso de la funcion de `predictOneVsAll()` creada anteriormente, los siguiente fue normalizar nuestra `X_test` que son el 20% para pruebas, ahora se procedio a normalizarlo haciendo uso de *mu* y *sigma* calculado anteriormente en la funcion de normalizacion.

luego se hace uso de `np.mean` donde nos calcula el promedio de los valores booleanos en el array resultante de la comparación. 

#### 1.6.1 Con los datos de entrenamiento
Para este caso se uso los datos de `X_norm` usando la funcion de `predictOneVsAll()`, viendo el resultado de precision nos da un **62%** el cual esta muy lejos de ser un buen modelo.

In [18]:
# print(X_test.shape)
pred_test = predictOneVsAll(all_theta, X_norm)
print('Precision del conjunto de entrenamiento: {:.2f}%'.format(np.mean(pred_test == y_train) * 100))

Precision del conjunto de entrenamiento: 62.00%


#### 1.6.2 con los datos de prueba
Para este caso se uso los datos de `X_test`, luego se procedio a normalizar los datos. 

Usando la funcion de `predictOneVsAll()`, viendo el resultado de precision nos da un **62.24%** , un poco en diferencia a la anterior prediccion.

In [19]:
X_test1 = X_test.copy()
# print(X_test.shape)
X_test = (X_test - mu) / sigma
pred_train = predictOneVsAll(all_theta, X_test)
print('Precision del conjunto de entrenamiento: {:.2f}%'.format(np.mean(pred_train == y_test) * 100))

Precision del conjunto de entrenamiento: 62.24%


En este caso se puede hacer las predicciones usando directamente la funcion `sigmoid()`, y luego imprimimos 30 predicciones para ver que tanto se asemejan con nuestra Y predicha.

In [20]:

#usando el mu y sigma calculado anteriormente, para realizar la normalizacion de los datos de prueba
X_test1 = (X_test1 - mu) / sigma
X_test1 = np.concatenate([np.ones((len(X_test1), 1)), X_test1], axis=1)

print(X_test1.shape)

#para este ejemplo, vamos a mostrar las predicciones de las primeras 30 predicciones
p = np.argmax(sigmoid(X_test1.dot(all_theta.T)), axis = 1)

for i in range(30):
    print(f"Prediccion: {p[i]}, Real: {y_test[i]}")

(15962, 24)
Prediccion: 0, Real: 1
Prediccion: 2, Real: 2
Prediccion: 2, Real: 2
Prediccion: 0, Real: 1
Prediccion: 1, Real: 1
Prediccion: 2, Real: 1
Prediccion: 2, Real: 1
Prediccion: 1, Real: 1
Prediccion: 1, Real: 1
Prediccion: 1, Real: 1
Prediccion: 0, Real: 1
Prediccion: 0, Real: 0
Prediccion: 0, Real: 0
Prediccion: 1, Real: 2
Prediccion: 1, Real: 1
Prediccion: 1, Real: 2
Prediccion: 2, Real: 2
Prediccion: 1, Real: 0
Prediccion: 1, Real: 0
Prediccion: 1, Real: 0
Prediccion: 1, Real: 0
Prediccion: 0, Real: 0
Prediccion: 1, Real: 2
Prediccion: 1, Real: 0
Prediccion: 1, Real: 1
Prediccion: 1, Real: 1
Prediccion: 1, Real: 1
Prediccion: 1, Real: 1
Prediccion: 0, Real: 0
Prediccion: 0, Real: 1
